In [74]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" #Only CPU
import tensorflow as tf
import numpy as np
import pathlib

In [75]:
DataSet_Path = '/Users/zhuzhirui/.keras/datasets/104_Flowers'
#Download Link https://www.kaggle.com/datasets/msheriey/104-flowers-garden-of-eden?resource=download


def Create_Class_List(path):
    '''
    To know how many classes in subdirectory
    Input: path  file path
    Output: class list
    '''
    File_Names = tf.io.gfile.glob(path + '/*')
    Class_List = tf.strings.regex_replace(File_Names, (path+'/'), '')
    Class = [item.numpy().decode("utf-8") for item in Class_List]
    Class = [item for item in Class if item.find(".") == -1]
    return Class

DataSet_Path = '/Users/zhuzhirui/.keras/datasets/104_Flowers'
#Download Link https://www.kaggle.com/datasets/msheriey/104-flowers-garden-of-eden?resource=download

Size_Type = Create_Class_List(DataSet_Path)
print(Size_Type)

for item in Size_Type:
    train_path = DataSet_Path + '/'+ item + '/train'
    val_path = DataSet_Path + '/'+ item + '/val'
    Training_Class = Create_Class_List(train_path)
    Val_Class = Create_Class_List(val_path)
    print('Train {} Class, Val {} Class --{}'.format(len(Training_Class), len(Val_Class), item))
    print('Train Class: ', Training_Class)
    print('--------------------------------------------------------------')
    print('Val Class: ', Val_Class)
    print('--------------------------------------------------------------')
    
CLASS = Training_Class #As same as any Training_Class or Val_Class
print(len(CLASS))

['jpeg-224x224', 'jpeg-512x512', 'jpeg-311x311', 'jpeg-192x192']
Train 104 Class, Val 104 Class --jpeg-224x224
Train Class:  ['toad lily', 'love in the mist', 'monkshood', 'azalea', 'fritillary', 'silverbush', 'canterbury bells', 'stemless gentian', 'pink primrose', 'buttercup', 'poinsettia', 'desert-rose', 'bird of paradise', 'columbine', 'frangipani', 'sweet pea', 'siam tulip', 'great masterwort', 'hard-leaved pocket orchid', 'marigold', 'foxglove', 'wild pansy', 'windflower', 'daisy', 'tiger lily', 'purple coneflower', 'orange dahlia', 'globe-flower', 'lilac hibiscus', 'fire lily', 'balloon flower', 'iris', 'bishop of llandaff', 'yellow iris', 'garden phlox', 'alpine sea holly', 'geranium', 'pink quill', 'tree poppy', 'spear thistle', 'bromelia', 'common dandelion', 'sword lily', 'peruvian lily', 'carnation', 'cosmos', 'spring crocus', 'lotus', 'bolero deep blue', 'anthurium', 'rose', 'water lily', 'primula', 'blackberry lily', 'gaura', 'trumpet creeper', 'globe thistle', 'sweet wil

In [97]:
def file_convert_numpylist(size_type, flag='train', is_random=True):
    '''
    Transform files information into numpy list
    Input: image size class, train or val, whether need random
            1 Size: jpeg-224x224/jpeg-512x512/jpeg-311x311/jpeg-192x192
            1 Flag: train/val
            104class = CLASS
    Output: labellist --> all images classes labels between [0,103]
            imagepath --> all images file paths
            idlist --> all images files names
    
    '''
    label_list = []
    image_path = []
    id_list = []

    for index in range(104):
        '''iterate 104 classes of one image size'''
        instead_path = DataSet_Path + '/' + size_type + '/' + flag + '/' + CLASS[index] +'/'
        image = tf.io.gfile.glob(instead_path +'*') #a list of all images path of one class
        
        image_path.extend(image) #add all images paths, every time one class
        
        image_id = tf.strings.regex_replace(image, instead_path, '') #extract all image names of one class
        image_id = np.asarray(image_id) # String Tensor like <tf.Tensor: shape=(), dtype=string, numpy=b'1451.jpeg'>

        id_list.extend(image_id)#one class every time
        #id_list = [item.numpy().decode("utf-8") for item in id_list]

        for index_label in range(len(image)): #one class every time
            label_list.append(index)
        
    label_list = np.asarray(label_list)
    image_path = np.asarray(image_path)
    id_list = np.asarray(id_list)
    
    if is_random:
        list_index = np.arange(len(id_list))
        np.random.shuffle(list_index)
        label_list = label_list[list_index]
        image_path = image_path[list_index]
        id_list = id_list[list_index]
    
    return label_list, image_path, id_list
    

In [98]:
#encode int64, bytes, float
def int64_feature(values):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))

def bytes_feature(values):
    #if isinstance(values,type(tf.constant(0))):
        #values = values.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))

def float_feature(values):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[values]))

def image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()])
        #transform Uint8 Tensor image into string Tensor, and numpy function transform into byte
    )

def image_to_tfexample(image, label, image_id):
    feature = {
        'label':int64_feature(label),
        'image':image_feature(image),
        'image_id':bytes_feature(image_id)
    }
    return tf.train.Features(feature=feature)

def convert_TFRFiles(label_list, image_path, id_list, TFRecord_filename):
    with tf.io.TFRecordWriter(TFRecord_filename) as writer:
        for i in range(len(image_path)):
            #image = open(image_list[i], 'rb').read()
            image = tf.image.decode_jpeg(tf.io.read_file(image_path[i]))#transform image into Uint8 Tensor
            label = label_list[i]
            image_id = id_list[i]
            example = tf.train.Example(features = image_to_tfexample(image, label, image_id))
            writer.write(example.SerializeToString())
            print('writing {} of {}'.format(i+1,len(image_path)))
        print('saved in {}'.format(TFRecord_filename))


In [99]:
for item in Size_Type:
    '''one size every time'''
    label_list, image_path, id_list = file_convert_numpylist(item, flag='train', is_random=True)
    TFRecord_filename = DataSet_Path+ '/'+ item + '_train.tfrecord'
    convert_TFRFiles(label_list, image_path, id_list, TFRecord_filename)
    
    label_list, image_path, id_list = file_convert_numpylist(item, flag='val', is_random=True)
    TFRecord_filename = DataSet_Path+ '/'+ item + '_val.tfrecord'
    convert_TFRFiles(label_list, image_path, id_list, TFRecord_filename)

writing 1 of 12753
writing 2 of 12753
writing 3 of 12753
writing 4 of 12753
writing 5 of 12753
writing 6 of 12753
writing 7 of 12753
writing 8 of 12753
writing 9 of 12753
writing 10 of 12753
writing 11 of 12753
writing 12 of 12753
writing 13 of 12753
writing 14 of 12753
writing 15 of 12753
writing 16 of 12753
writing 17 of 12753
writing 18 of 12753
writing 19 of 12753
writing 20 of 12753
writing 21 of 12753
writing 22 of 12753
writing 23 of 12753
writing 24 of 12753
writing 25 of 12753
writing 26 of 12753
writing 27 of 12753
writing 28 of 12753
writing 29 of 12753
writing 30 of 12753
writing 31 of 12753
writing 32 of 12753
writing 33 of 12753
writing 34 of 12753
writing 35 of 12753
writing 36 of 12753
writing 37 of 12753
writing 38 of 12753
writing 39 of 12753
writing 40 of 12753
writing 41 of 12753
writing 42 of 12753
writing 43 of 12753
writing 44 of 12753
writing 45 of 12753
writing 46 of 12753
writing 47 of 12753
writing 48 of 12753
writing 49 of 12753
writing 50 of 12753
writing 5

writing 433 of 12753
writing 434 of 12753
writing 435 of 12753
writing 436 of 12753
writing 437 of 12753
writing 438 of 12753
writing 439 of 12753
writing 440 of 12753
writing 441 of 12753
writing 442 of 12753
writing 443 of 12753
writing 444 of 12753
writing 445 of 12753
writing 446 of 12753
writing 447 of 12753
writing 448 of 12753
writing 449 of 12753
writing 450 of 12753
writing 451 of 12753
writing 452 of 12753
writing 453 of 12753
writing 454 of 12753
writing 455 of 12753
writing 456 of 12753
writing 457 of 12753
writing 458 of 12753
writing 459 of 12753
writing 460 of 12753
writing 461 of 12753
writing 462 of 12753
writing 463 of 12753
writing 464 of 12753
writing 465 of 12753
writing 466 of 12753
writing 467 of 12753
writing 468 of 12753
writing 469 of 12753
writing 470 of 12753
writing 471 of 12753
writing 472 of 12753
writing 473 of 12753
writing 474 of 12753
writing 475 of 12753
writing 476 of 12753
writing 477 of 12753
writing 478 of 12753
writing 479 of 12753
writing 480 o

writing 842 of 12753
writing 843 of 12753
writing 844 of 12753
writing 845 of 12753
writing 846 of 12753
writing 847 of 12753
writing 848 of 12753
writing 849 of 12753
writing 850 of 12753
writing 851 of 12753
writing 852 of 12753
writing 853 of 12753
writing 854 of 12753
writing 855 of 12753
writing 856 of 12753
writing 857 of 12753
writing 858 of 12753
writing 859 of 12753
writing 860 of 12753
writing 861 of 12753
writing 862 of 12753
writing 863 of 12753
writing 864 of 12753
writing 865 of 12753
writing 866 of 12753
writing 867 of 12753
writing 868 of 12753
writing 869 of 12753
writing 870 of 12753
writing 871 of 12753
writing 872 of 12753
writing 873 of 12753
writing 874 of 12753
writing 875 of 12753
writing 876 of 12753
writing 877 of 12753
writing 878 of 12753
writing 879 of 12753
writing 880 of 12753
writing 881 of 12753
writing 882 of 12753
writing 883 of 12753
writing 884 of 12753
writing 885 of 12753
writing 886 of 12753
writing 887 of 12753
writing 888 of 12753
writing 889 o

writing 1239 of 12753
writing 1240 of 12753
writing 1241 of 12753
writing 1242 of 12753
writing 1243 of 12753
writing 1244 of 12753
writing 1245 of 12753
writing 1246 of 12753
writing 1247 of 12753
writing 1248 of 12753
writing 1249 of 12753
writing 1250 of 12753
writing 1251 of 12753
writing 1252 of 12753
writing 1253 of 12753
writing 1254 of 12753
writing 1255 of 12753
writing 1256 of 12753
writing 1257 of 12753
writing 1258 of 12753
writing 1259 of 12753
writing 1260 of 12753
writing 1261 of 12753
writing 1262 of 12753
writing 1263 of 12753
writing 1264 of 12753
writing 1265 of 12753
writing 1266 of 12753
writing 1267 of 12753
writing 1268 of 12753
writing 1269 of 12753
writing 1270 of 12753
writing 1271 of 12753
writing 1272 of 12753
writing 1273 of 12753
writing 1274 of 12753
writing 1275 of 12753
writing 1276 of 12753
writing 1277 of 12753
writing 1278 of 12753
writing 1279 of 12753
writing 1280 of 12753
writing 1281 of 12753
writing 1282 of 12753
writing 1283 of 12753
writing 12

writing 1658 of 12753
writing 1659 of 12753
writing 1660 of 12753
writing 1661 of 12753
writing 1662 of 12753
writing 1663 of 12753
writing 1664 of 12753
writing 1665 of 12753
writing 1666 of 12753
writing 1667 of 12753
writing 1668 of 12753
writing 1669 of 12753
writing 1670 of 12753
writing 1671 of 12753
writing 1672 of 12753
writing 1673 of 12753
writing 1674 of 12753
writing 1675 of 12753
writing 1676 of 12753
writing 1677 of 12753
writing 1678 of 12753
writing 1679 of 12753
writing 1680 of 12753
writing 1681 of 12753
writing 1682 of 12753
writing 1683 of 12753
writing 1684 of 12753
writing 1685 of 12753
writing 1686 of 12753
writing 1687 of 12753
writing 1688 of 12753
writing 1689 of 12753
writing 1690 of 12753
writing 1691 of 12753
writing 1692 of 12753
writing 1693 of 12753
writing 1694 of 12753
writing 1695 of 12753
writing 1696 of 12753
writing 1697 of 12753
writing 1698 of 12753
writing 1699 of 12753
writing 1700 of 12753
writing 1701 of 12753
writing 1702 of 12753
writing 17

...

writing 1624 of 3712
writing 1625 of 3712
writing 1626 of 3712
writing 1627 of 3712
writing 1628 of 3712
writing 1629 of 3712
writing 1630 of 3712
writing 1631 of 3712
writing 1632 of 3712
writing 1633 of 3712
writing 1634 of 3712
writing 1635 of 3712
writing 1636 of 3712
writing 1637 of 3712
writing 1638 of 3712
writing 1639 of 3712
writing 1640 of 3712
writing 1641 of 3712
writing 1642 of 3712
writing 1643 of 3712
writing 1644 of 3712
writing 1645 of 3712
writing 1646 of 3712
writing 1647 of 3712
writing 1648 of 3712
writing 1649 of 3712
writing 1650 of 3712
writing 1651 of 3712
writing 1652 of 3712
writing 1653 of 3712
writing 1654 of 3712
writing 1655 of 3712
writing 1656 of 3712
writing 1657 of 3712
writing 1658 of 3712
writing 1659 of 3712
writing 1660 of 3712
writing 1661 of 3712
writing 1662 of 3712
writing 1663 of 3712
writing 1664 of 3712
writing 1665 of 3712
writing 1666 of 3712
writing 1667 of 3712
writing 1668 of 3712
writing 1669 of 3712
writing 1670 of 3712
writing 1671 

writing 2110 of 3712
writing 2111 of 3712
writing 2112 of 3712
writing 2113 of 3712
writing 2114 of 3712
writing 2115 of 3712
writing 2116 of 3712
writing 2117 of 3712
writing 2118 of 3712
writing 2119 of 3712
writing 2120 of 3712
writing 2121 of 3712
writing 2122 of 3712
writing 2123 of 3712
writing 2124 of 3712
writing 2125 of 3712
writing 2126 of 3712
writing 2127 of 3712
writing 2128 of 3712
writing 2129 of 3712
writing 2130 of 3712
writing 2131 of 3712
writing 2132 of 3712
writing 2133 of 3712
writing 2134 of 3712
writing 2135 of 3712
writing 2136 of 3712
writing 2137 of 3712
writing 2138 of 3712
writing 2139 of 3712
writing 2140 of 3712
writing 2141 of 3712
writing 2142 of 3712
writing 2143 of 3712
writing 2144 of 3712
writing 2145 of 3712
writing 2146 of 3712
writing 2147 of 3712
writing 2148 of 3712
writing 2149 of 3712
writing 2150 of 3712
writing 2151 of 3712
writing 2152 of 3712
writing 2153 of 3712
writing 2154 of 3712
writing 2155 of 3712
writing 2156 of 3712
writing 2157 

writing 2589 of 3712
writing 2590 of 3712
writing 2591 of 3712
writing 2592 of 3712
writing 2593 of 3712
writing 2594 of 3712
writing 2595 of 3712
writing 2596 of 3712
writing 2597 of 3712
writing 2598 of 3712
writing 2599 of 3712
writing 2600 of 3712
writing 2601 of 3712
writing 2602 of 3712
writing 2603 of 3712
writing 2604 of 3712
writing 2605 of 3712
writing 2606 of 3712
writing 2607 of 3712
writing 2608 of 3712
writing 2609 of 3712
writing 2610 of 3712
writing 2611 of 3712
writing 2612 of 3712
writing 2613 of 3712
writing 2614 of 3712
writing 2615 of 3712
writing 2616 of 3712
writing 2617 of 3712
writing 2618 of 3712
writing 2619 of 3712
writing 2620 of 3712
writing 2621 of 3712
writing 2622 of 3712
writing 2623 of 3712
writing 2624 of 3712
writing 2625 of 3712
writing 2626 of 3712
writing 2627 of 3712
writing 2628 of 3712
writing 2629 of 3712
writing 2630 of 3712
writing 2631 of 3712
writing 2632 of 3712
writing 2633 of 3712
writing 2634 of 3712
writing 2635 of 3712
writing 2636 

writing 2982 of 3712
writing 2983 of 3712
writing 2984 of 3712
writing 2985 of 3712
writing 2986 of 3712
writing 2987 of 3712
writing 2988 of 3712
writing 2989 of 3712
writing 2990 of 3712
writing 2991 of 3712
writing 2992 of 3712
writing 2993 of 3712
writing 2994 of 3712
writing 2995 of 3712
writing 2996 of 3712
writing 2997 of 3712
writing 2998 of 3712
writing 2999 of 3712
writing 3000 of 3712
writing 3001 of 3712
writing 3002 of 3712
writing 3003 of 3712
writing 3004 of 3712
writing 3005 of 3712
writing 3006 of 3712
writing 3007 of 3712
writing 3008 of 3712
writing 3009 of 3712
writing 3010 of 3712
writing 3011 of 3712
writing 3012 of 3712
writing 3013 of 3712
writing 3014 of 3712
writing 3015 of 3712
writing 3016 of 3712
writing 3017 of 3712
writing 3018 of 3712
writing 3019 of 3712
writing 3020 of 3712
writing 3021 of 3712
writing 3022 of 3712
writing 3023 of 3712
writing 3024 of 3712
writing 3025 of 3712
writing 3026 of 3712
writing 3027 of 3712
writing 3028 of 3712
writing 3029 

writing 3500 of 3712
writing 3501 of 3712
writing 3502 of 3712
writing 3503 of 3712
writing 3504 of 3712
writing 3505 of 3712
writing 3506 of 3712
writing 3507 of 3712
writing 3508 of 3712
writing 3509 of 3712
writing 3510 of 3712
writing 3511 of 3712
writing 3512 of 3712
writing 3513 of 3712
writing 3514 of 3712
writing 3515 of 3712
writing 3516 of 3712
writing 3517 of 3712
writing 3518 of 3712
writing 3519 of 3712
writing 3520 of 3712
writing 3521 of 3712
writing 3522 of 3712
writing 3523 of 3712
writing 3524 of 3712
writing 3525 of 3712
writing 3526 of 3712
writing 3527 of 3712
writing 3528 of 3712
writing 3529 of 3712
writing 3530 of 3712
writing 3531 of 3712
writing 3532 of 3712
writing 3533 of 3712
writing 3534 of 3712
writing 3535 of 3712
writing 3536 of 3712
writing 3537 of 3712
writing 3538 of 3712
writing 3539 of 3712
writing 3540 of 3712
writing 3541 of 3712
writing 3542 of 3712
writing 3543 of 3712
writing 3544 of 3712
writing 3545 of 3712
writing 3546 of 3712
writing 3547 

In [96]:
image_path = []
label_list = []
id_list = []

instead_path = DataSet_Path + '/' + Size_Type[0] + '/' + 'train' + '/' + CLASS[0] +'/'
image = tf.io.gfile.glob(instead_path +'*') #a list of all images path of one class

image_path.extend(image) #add all images paths, every time one class

image_id = tf.strings.regex_replace(image, instead_path, '') #extract all image names of one class
#image_id = np.array(image_id) # String Tensor like <tf.Tensor: shape=(), dtype=string, numpy=b'1451.jpeg'>

id_list.extend(image_id)#one class every time
#id_list = [item.numpy().decode("utf-8") for item in id_list]

for index_label in range(len(image)): #one class every time
    label_list.append(0)
    
print(id_list)

instead_path = DataSet_Path + '/' + Size_Type[0] + '/' + 'train' + '/' + CLASS[1] +'/'
image = tf.io.gfile.glob(instead_path +'*') #a list of all images path of one class

image_path.extend(image) #add all images paths, every time one class

image_id = tf.strings.regex_replace(image, instead_path, '') #extract all image names of one class
#image_id = np.asarray(image_id) # String Tensor like <tf.Tensor: shape=(), dtype=string, numpy=b'1451.jpeg'>

id_list.extend(image_id)#one class every time
#id_list = [item.numpy().decode("utf-8") for item in id_list]

for index_label in range(len(image)): #one class every time
    label_list.append(1)
    
print(id_list)

[<tf.Tensor: shape=(), dtype=string, numpy=b'631.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'8433.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'11287.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'2815.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'2474.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'4735.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'2177.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'2874.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'781.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'4430.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'3530.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'1598.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'3993.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'8330.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'8023.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'9455.jpeg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'5437.jpeg'>, <tf.Tensor: shape=(), dtype=str